In [106]:
using Plots, Distributions
include("./src/supportFunctions.ji")
include("./src/qgplsim.ji")
pwd()

"/home/kyan/Projects/qgplsim"

# Case 1
$y = X \alpha + exp(X \theta) + \epsilon$

In [5]:
function simu_data1(X, α, θ, ϵ)

    y = X*α + exp.(X*θ) + ϵ

end

## test 

In [109]:
∑ = zeros(5,5)

for i in 1:5
    for j in i:5
        ∑[i, j] = 0.5^(j - i)
        ∑[j, i] = ∑[i, j]
    end
end

∑ = sqrt.(∑)

n = 200
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3

U = randn(n, 5)
X = U * ∑
e = rand(n) / 2
y = simu_data1(X, alpha, theta, e)
Z = zeros(n, 1)

tau = [0.5]
widthExp = -0.17
model1 = qgplsim.model(X, Z, y, tau, widthExp)
qgplsim.estimator(model1, "optim_qr")
qgplsim.print_model(model1)

alpha => [0.1431, 0.993, -0.0515, -2.0234, 0.1335]
theta => [0.6977, -0.0542, -0.3425, 0.0725, 0.6227]
gamma => [0.0]
beta => [0.0]


## Monte Carlo Simulation

In [110]:
using Base.Threads
nth = Threads.nthreads()
println("threads for parallel computing:  ", nth)

threads for parallel computing:  4


In [111]:
n = 200
alpha = [0, 1, 0, -2, 0]
theta = [2, 0, -1, 0, 2]/3
Z = zeros(n, 1)
n_repeat = 25
n_repeat * nth

100

### $\epsilon$ ~ $N(0, 1)/2$

In [83]:
Alpha1 = zeros(n_repeat * nth, 5)
Theta1 = zeros(n_repeat * nth, 5)
@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(n) / 2
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha1[k, :] = modelk.alpha
        Theta1[k, :] = modelk.theta
    end
end



In [102]:
EE1 = sqrt.(1 .- abs.(Theta1 * theta)) 
println(mean(EE1), " ", std(EE1))

0.16482091958569153 0.1980300839525535


In [85]:
mean(Theta1, dims = 1)

1×5 Matrix{Float64}:
 0.644122  0.0373451  -0.309425  -0.00394015  0.589856

### $\epsilon$ ~ $\sqrt 3 t(3) / 10$

In [86]:
t3 = TDist(3)

TDist{Float64}(ν=3.0)

In [87]:
Alpha2 = zeros(n_repeat * nth, 5)
Theta2 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(t3, n) * sqrt(3) / 10
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha2[k, :] = modelk.alpha
        Theta2[k, :] = modelk.theta
    end
end

In [88]:
mean(Theta2, dims = 1)

1×5 Matrix{Float64}:
 0.621185  0.0157167  -0.286917  -0.0121791  0.555159

In [101]:
EE2 = sqrt.(1 .- abs.(Theta2 * theta)) 
println(mean(EE2), " ", std(EE2))

0.19210274022474047 0.22122747635563317


### $\epsilon$ ~ $t(1)/10$

In [90]:
t1 = TDist(1)

TDist{Float64}(ν=1.0)

In [91]:
Alpha3 = zeros(n_repeat * nth, 5)
Theta3 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = rand(t1, n) / 10
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha3[k, :] = modelk.alpha
        Theta3[k, :] = modelk.theta
    end
end

In [92]:
mean(Theta3, dims = 1)

1×5 Matrix{Float64}:
 0.663367  -0.00342113  -0.337264  0.0275873  0.611017

In [100]:
EE3 = sqrt.(1 .- abs.(Theta3 * theta)) 
println(mean(EE3), " ", std(EE3))

0.1307498044544481 0.11712713930977704


### $\epsilon$ ~ $0.3(N(0, 1)^4 - 3)$

In [94]:
Alpha4 = zeros(n_repeat * nth, 5)
Theta4 = zeros(n_repeat * nth, 5)

@threads for i in 1:nth
    for j in 1:n_repeat
        
        k = (i - 1) * n_repeat + j
        U = randn(n, 5)
        X = U * ∑
        e = randn(n)
        e = (e.^4 .- 3) .* 0.3
        y = simu_data1(X, alpha, theta, e)
        modelk = qgplsim.model(X, Z, y, tau, widthExp)
        qgplsim.estimator(modelk, "optim_qr")
        Alpha4[k, :] = modelk.alpha
        Theta4[k, :] = modelk.theta
    end
end

In [95]:
mean(Theta4, dims = 1)

1×5 Matrix{Float64}:
 0.647749  -0.010852  -0.357562  0.0789847  0.583949

In [99]:
EE4 = sqrt.(1 .- abs.(Theta4 * theta)) 
println(mean(EE4), " ", std(EE4))

0.20204432434805344 0.1367672696927415
